In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import os

def read_binarry(file_path):
    import struct
    import math

    power = []
    age = []
    coordinate_x = []
    coordinate_y = []
    angle_tetta = []
    angle_phi = []
    energy = []
    time = []

    with open(file_path, 'rb') as binary_file:
        for i in range(100000):
            binary_file.read(4 * 5)

            tetta = struct.unpack('f', binary_file.read(4))[0]
            angle_tetta.append(tetta)

            phi = struct.unpack('f', binary_file.read(4))[0]
            angle_phi.append(phi)

            x0 = struct.unpack('f', binary_file.read(4))[0]
            coordinate_x.append(x0)

            y0 = struct.unpack('f', binary_file.read(4))[0]
            coordinate_y.append(y0)

            binary_file.read(4 * 5)

            power_eas = struct.unpack('f', binary_file.read(4))[0]
            power.append(math.log10(power_eas))

            age_eas = struct.unpack('f', binary_file.read(4))[0]
            age.append(age_eas)

            binary_file.read(4 * 1565)
            energy_release = struct.unpack('f' * 36, binary_file.read(4 * 36))
            energy.append(energy_release)

            binary_file.read(4)
            t = struct.unpack('f' * 144, binary_file.read(4 * 144))
            threshold_time = t[::4]
            time.append(threshold_time)

    # Собираем всё в DataFrame
    df = pd.DataFrame({
        'power': power,
        'age': age,
        'x': coordinate_x,
        'y': coordinate_y,
        'tetta': angle_tetta,
        'phi': angle_phi,
        'energy': energy,
        'threshold_time': time,
    })

    return df

In [ ]:
#Заморозка
def set_seed(seed: int = 42) -> None:
    np.random.seed(seed)
    os.environ["PYTHONHASHSEED"] = str(seed)

set_seed(42)
random_state = 42

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [19]:
df = read_binarry("/content/drive/MyDrive/data.dat")

In [21]:
new_columns = pd.DataFrame(
    df['energy'].tolist(),
    columns=[f'energy_{i}' for i in range(36)]
)

df = pd.concat([df, new_columns], axis=1)

In [39]:
new_columns = pd.DataFrame(
    df['threshold_time'].tolist(),
    columns=[f'threshold_time_{i}' for i in range(36)]
)

df = pd.concat([df, new_columns], axis=1)

KeyError: 'threshold_time'

In [ ]:
df.drop(columns=["energy", "threshold_time"], inplace=True)

In [40]:
X = df.drop(columns=["power", "age", "x", "y", "tetta", "phi"])

In [41]:
import math
X = df.drop(columns=["power", "age", "x", "y", "tetta", "phi"])
y = (df['tetta']*math.pi/180).agg('cos') #pd.concat([prep1,prep2], axis=1)

In [43]:
import lightgbm as lgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.datasets import fetch_california_housing

In [44]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=random_state)

In [45]:
train_data = lgb.Dataset(X_train, label=y_train)
test_data = lgb.Dataset(X_test, label=y_test, reference=train_data)

In [46]:
params = {
    'objective': 'regression',       # Тип задачи - регрессия
    'metric': 'rmse',                # Метрика качества (Root Mean Squared Error)
    'boosting_type': 'gbdt',         # Алгоритм градиентного бустинга
    'num_leaves': 31,                # Максимальное количество листьев в дереве
    'learning_rate': 0.05,           # Скорость обучения
    'feature_fraction': 0.9,         # Доля случайно выбираемых признаков на каждой итерации
    'bagging_fraction': 0.8,         # Доля данных для бутстрепа
    'bagging_freq': 6,               # Частота бэггинга
    'verbose': 1,                    # Отключение выводов
    'n_jobs': 1,                    # Использовать все ядра процессора
    'random_state': 42               # Для воспроизводимости
}

In [47]:
model = lgb.train(
    params,
    train_data,
    num_boost_round=100,            # Максимальное количество деревьев
    valid_sets=[train_data, test_data],
    valid_names=['train', 'valid']
)

[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.085963 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 18360
[LightGBM] [Info] Number of data points in the train set: 80000, number of used features: 72
[LightGBM] [Info] Start training from score 0.821163


In [ ]:
y_predscos = model.predict(X_test, num_iteration=model.best_iteration)

In [ ]:
y_predscos

array([0.90500275, 0.86628673, 0.7532067 , ..., 0.85325918, 0.86735008,
       0.68815764])

In [ ]:
def evaluate_regression(y_true, y_pred):
    rmse = np.sqrt(mean_squared_error(y_true, y_pred))
    mae = mean_absolute_error(y_true, y_pred)
    r2 = r2_score(y_true, y_pred)

    print(f"RMSE: {rmse:.4f}")
    print(f"MAE: {mae:.4f}")
    print(f"R²: {r2:.4f}")
    return {'rmse': rmse, 'mae': mae, 'r2': r2}

metrics = evaluate_regression(y_test, y_predscos)

RMSE: 0.0668
MAE: 0.0533
R²: 0.5818


In [ ]:
np.degrees(np.arccos(model.predict(X_test, num_iteration=model.best_iteration)))

/tmp/ipython-input-157-1578889871.py:1: RuntimeWarning: invalid value encountered in arccos
  np.degrees(np.arccos(model.predict(X_test, num_iteration=model.best_iteration)))


array([ 96.7056368 , 109.46072153, 119.86466162, ...,  99.78845538,
       102.14111386,  89.62261295])